In [1]:
import sys
sys.path.append(r"C:\Users\mauro\Downloads\mido-master")
sys.path.append(r"C:\Users\mauro\Downloads\pretty-midi-master")

import mido
import pretty_midi
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
files = []
#Grabs files 
for entry in os.scandir('Midi_files'):
    if entry.is_file():
        files.append(entry.path)

In [3]:
duration = []
end_time = []
#determines timestamp
for midi in files:
    midi_data = pretty_midi.PrettyMIDI(midi)
    for instrument in midi_data.instruments:
        for i,note in enumerate(instrument.notes):
                duration.append(note.end - note.start)
        end_time.append(note.end)
        break
duration.sort()
bucket_length = int(len(duration)/4)
duration_bucket_1 = bucket_length
duration_bucket_2 = bucket_length * 2
duration_bucket_3 = bucket_length * 3

step = duration[duration_bucket_1]
step = round(step,4)


In [4]:
import numpy as np
import tensorflow as tf
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [5]:
song = []
#grabs notes from songs
for midi in files:
    current_step = 0.0
    current_index = 0
    midi_data = pretty_midi.PrettyMIDI(midi)

    note = []
    for instrument in midi_data.instruments:
        if(instrument.name=="Drum kit 2"):
            continue
        if not instrument.is_drum:
            for n in instrument.notes:
                #determines duration lenght and timestamp bucket for each
                time = n.end - n.start
                if (time < duration[duration_bucket_1]):
                    time = 1
                elif (time < duration[duration_bucket_2]):
                    time = 2
                elif (time < duration[duration_bucket_3]):
                    time = 3
                else:
                    time = 4
                
                
                note.append(time)
                note.append(n.pitch)
                note.append(n.velocity)
                
                #converts note into one value string
                song.append('.'.join(str(i) for i in note))
                note.clear()
               
            break
#number of note types
n_vocab = len(set(song))
#collection of each note_type
pitchnames = sorted(set(item for item in song))   

#gives an int value to each note type
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

#main input vector
network_input = []
#output vector
network_output = []
#notes in sequence
sequence_length = 100

#sets up the list of 100 note sequences
for i in range(0, len(song) - sequence_length, 1):
    sequence_in = song[i:i + sequence_length]
    sequence_out = song[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
#how many total sequences    
n_patterns = len(network_input)

#converts to lstm readable matrix
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
#normalizes note values
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

print(network_input.shape)

(8164, 100, 1)


In [6]:
# Building the model architecture
#network_input.shape=(time, pitch, velocity)
model = Sequential()  
model.add(LSTM(1024, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))  
model.add(Dropout(0.2))                                                # prevents overfitting
model.add(LSTM(1024, return_sequences=True))  
model.add(Flatten())  
model.add(Dense(512))  
model.add(Dropout(0.3))  
model.add(Dense(n_vocab))                                              # each input node becomes connected to output node
model.add(Activation('softmax'))  
model.compile(loss='categorical_crossentropy', optimizer='adam')    # optimizer= how it learns while loss= how well it learns

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 1024)         8392704   
_________________________________________________________________
flatten_1 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               52429312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1351)              693063    
__________

In [ ]:
from keras.callbacks import ModelCheckpoint
    
# Create checkpoint to save the best model weights.
filepath = filepath = r"C:\\Users\\mauro\\Downloads\\weights-improvement--{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=[checkpoint])

In [ ]:
# ignore from here down until top works
'''
# Building the model architecture
model = Sequential()  
model.add(LSTM(1024, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))  
model.add(Dropout(0.2))                                                # prevents overfitting
model.add(LSTM(1024, return_sequences=True))  
model.add(Flatten())  
model.add(Dense(512))  
model.add(Dropout(0.3))  
model.add(Dense(n_vocab))                                              # each input node becomes connected to output node
model.add(Activation('softmax'))  
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')    # optimizer= how it learns while loss= how well it learns

model.load_weights(r"C:\\Users\\mauro\\Downloads\\weights-improvement-bigger.hdf5")
'''

In [ ]:
'''
def train_network():  
    epochs = 200 
    
    notes = get_notes()  
    print('Notes processed') 
    
    n_vocab = len(set(notes))  
    print('Vocab generated')
    
    # create input and output sequences
    network_in, network_out = prepare_sequences(notes, n_vocab)  
    print('Input and Output processed')  
    
    model = create_network(network_in, n_vocab)  
    print('Model created') 
    
    return model  

    print('Training in progress')  
    
    train(model, network_in, network_out, epochs)  
    print('Training completed')  
'''

In [ ]:
# Train model
#train_network()

In [ ]:
'''
def generate_notes(model, network_input, pitchnames, n_vocab):  
    """ Generate notes from the neural network based on a sequence of notes """  
   # Pick a random integer  
    start = np.random.randint(0, len(network_input)-1)  
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))  
    # pick a random sequence from the input as a starting point for the prediction  
    pattern = network_input[start]  
    prediction_output = []  
    print('Generating notes........')  
    # generate 500 notes  
    for note_index in range(500):  
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))  
        prediction_input = prediction_input / float(n_vocab)  
        prediction = model.predict(prediction_input, verbose=0)  
         # Predicted output is the argmax(P(h|D))  
        index = np.argmax(prediction)  
         # Mapping the predicted interger back to the corresponding note  
        result = int_to_note[index]  
         # Storing the predicted output  
        prediction_output.append(result)  
        pattern.append(index)  
         # Next input to the model  
        pattern = pattern[1:len(pattern)]  
    print('Notes Generated...')  
    return prediction_output  
'''

In [ ]:
 ### Converts the predicted output to midi format  
#create_midi(prediction_output)  
